<a href="https://colab.research.google.com/github/mariafernanda0011/Classificador-CNH/blob/main/trata_features_treina_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Features e Treinamento do Classificador de CNH**

## **Para testar os modelos treinados neste notebook acesse:**
- [Demonstração de modelos](https://colab.research.google.com/drive/1Kt7xidXbxcUheCST9o1aG3E_PquzltEs?usp=sharing)




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
features0 = pd.read_csv('features_cnh_digital_CNH_aberta.csv')
features1 = pd.read_csv('features_cnh_digital_CNH_frente.csv')
features2 = pd.read_csv('features_cnh_digital_CNH_verso.csv')

features3 = pd.read_csv('features_cnh_fisica_CNH_aberta.csv')
features4 = pd.read_csv('features_cnh_fisica_CNH_frente.csv')
features5 = pd.read_csv('features_cnh_fisica_CNH_verso.csv')

In [3]:
#Deletando colunas desnecessarias
features0 = features0.drop(['texto_extraido','categoria', 'subpasta'], axis=1)
features1 = features1.drop(['texto_extraido', 'categoria', 'subpasta'], axis=1)
features2 = features2.drop(['texto_extraido', 'categoria', 'subpasta'], axis=1)

features3 = features3.drop(['texto_extraido', 'categoria', 'subpasta'], axis=1)
features4 = features4.drop(['texto_extraido', 'categoria', 'subpasta'], axis=1)
features5 = features5.drop(['texto_extraido', 'categoria', 'subpasta'], axis=1)

In [4]:
#inserindo colunas com label
# A coluna 'label' é o alvo (0 ou 1)
features0['label'] = 0 # CNH digital aberta
features1['label'] = 1 # CNH digital frente
features2['label'] = 2 # CNH digital verso

features3['label'] = 3 # CNH fisica aberta
features4['label'] = 4 # CNH fisica frente
features5['label'] = 5 # CNH fisica verso

## **Exploração de dados - analisando as features**

In [5]:
# Vendo resultado das features da CNH digital aberta
features0.head()

,nome_arquivo,quantidade_palavras,qr_detectado,color1_h,color1_s,color1_v,color2_h,color2_s,color2_v,color3_h,...,portador,transportes,assinado,digital,estadual,infraestrutura,emissor,proibido,plastificar,label
0,aug_imagem_1.jpg,136,1,60,64,4,70,3,252,15,...,0,1,1,1,0,0,0,0,0,0
1,aug_imagem_10.jpg,127,1,60,43,6,60,2,251,18,...,0,1,1,1,0,0,0,0,0,0
2,aug_imagem_100.jpg,97,1,0,0,54,27,14,168,68,...,0,0,1,1,0,0,0,0,0,0
3,aug_imagem_101.jpg,132,1,60,51,5,70,3,252,15,...,0,0,1,1,0,0,0,0,0,0
4,aug_imagem_102.jpg,112,1,0,0,4,15,12,85,70,...,0,1,1,1,0,0,0,0,0,0


In [6]:
features0['digital'].value_counts()

,count
digital,
1,288


In [7]:
#vendo resultado
features1.head()

,nome_arquivo,quantidade_palavras,qr_detectado,color1_h,color1_s,color1_v,color2_h,color2_s,color2_v,color3_h,...,portador,transportes,assinado,digital,estadual,infraestrutura,emissor,proibido,plastificar,label
0,aug_imagem_1.jpg,41,0,60,32,8,63,10,240,17,...,0,0,0,0,0,0,0,0,0,1
1,aug_imagem_10.jpg,28,0,16,76,179,64,7,252,60,...,0,0,0,0,0,0,0,0,0,1
2,aug_imagem_100.jpg,32,0,17,75,171,66,10,246,60,...,0,0,0,0,0,0,0,0,0,1
3,aug_imagem_101.jpg,28,0,17,69,170,75,43,12,63,...,0,0,0,0,0,0,0,0,0,1
4,aug_imagem_102.jpg,30,0,17,75,171,70,70,11,66,...,0,0,0,0,0,0,0,0,0,1


In [8]:
features1['digital'].value_counts()

,count
digital,
0,288


In [9]:
features2.head()

,nome_arquivo,quantidade_palavras,qr_detectado,color1_h,color1_s,color1_v,color2_h,color2_s,color2_v,color3_h,...,portador,transportes,assinado,digital,estadual,infraestrutura,emissor,proibido,plastificar,label
0,aug_imagem_1.jpg,13,0,80,36,21,72,10,245,80,...,0,0,0,0,0,0,0,0,0,2
1,aug_imagem_10.jpg,5,0,90,16,48,74,11,245,80,...,0,0,0,0,0,0,0,0,0,2
2,aug_imagem_100.jpg,13,0,80,5,154,72,10,245,80,...,0,0,0,0,0,0,0,0,0,2
3,aug_imagem_101.jpg,18,0,83,6,177,74,11,245,80,...,0,0,0,0,0,0,0,0,0,2
4,aug_imagem_102.jpg,8,0,72,10,245,80,5,156,80,...,0,0,0,0,0,0,0,0,0,2


In [10]:
features2['digital'].value_counts()

,count
digital,
0,288


In [11]:
features3.head()

,nome_arquivo,quantidade_palavras,qr_detectado,color1_h,color1_s,color1_v,color2_h,color2_s,color2_v,color3_h,...,portador,transportes,assinado,digital,estadual,infraestrutura,emissor,proibido,plastificar,label
0,aug_imagem_1.jpg,91,0,27,49,120,22,28,171,0,...,1,0,0,0,0,0,1,0,0,3
1,aug_imagem_10.jpg,95,0,0,27,19,22,29,170,29,...,1,0,0,0,0,0,1,0,0,3
2,aug_imagem_100.jpg,0,0,26,32,137,0,7,37,23,...,0,0,0,0,0,0,0,0,0,3
3,aug_imagem_101.jpg,85,0,29,44,127,21,28,174,0,...,1,0,0,0,0,0,1,0,0,3
4,aug_imagem_102.jpg,70,0,29,41,130,0,24,21,21,...,1,0,0,0,0,0,1,0,0,3


In [12]:
features3['plastificar'].value_counts()

,count
plastificar,
0,288


In [13]:
features4.head()

,nome_arquivo,quantidade_palavras,qr_detectado,color1_h,color1_s,color1_v,color2_h,color2_s,color2_v,color3_h,...,portador,transportes,assinado,digital,estadual,infraestrutura,emissor,proibido,plastificar,label
0,aug_imagem_1.jpg,14,0,18,58,22,21,27,178,30,...,0,0,0,0,0,0,0,0,0,4
1,aug_imagem_10.jpg,0,0,30,28,45,21,22,187,22,...,0,0,0,0,0,0,0,0,0,4
2,aug_imagem_100.jpg,7,0,22,25,187,20,35,22,24,...,0,0,0,0,0,0,0,0,0,4
3,aug_imagem_101.jpg,78,0,30,42,134,20,33,46,21,...,0,0,0,0,0,0,0,0,0,4
4,aug_imagem_102.jpg,0,0,20,27,144,24,21,195,22,...,0,0,0,0,0,0,0,0,0,4


In [14]:
features5.head()

,nome_arquivo,quantidade_palavras,qr_detectado,color1_h,color1_s,color1_v,color2_h,color2_s,color2_v,color3_h,...,portador,transportes,assinado,digital,estadual,infraestrutura,emissor,proibido,plastificar,label
0,aug_imagem_1.jpg,0,0,35,25,184,28,25,130,23,...,0,0,0,0,0,0,0,0,0,5
1,aug_imagem_10.jpg,21,0,28,48,128,24,30,162,15,...,0,0,0,0,0,0,1,0,0,5
2,aug_imagem_100.jpg,10,0,20,70,11,27,64,112,27,...,0,0,0,0,0,0,1,0,0,5
3,aug_imagem_101.jpg,14,0,24,31,164,29,43,129,15,...,0,0,0,0,0,0,1,0,0,5
4,aug_imagem_102.jpg,24,0,27,52,112,25,32,145,12,...,0,0,0,0,0,0,1,0,0,5


In [15]:
#juntando dataframe em um só
features_completas = pd.concat([features0, features1, features2, features3, features4, features5])

In [16]:
#vendo features completas
features_completas.head()

,nome_arquivo,quantidade_palavras,qr_detectado,color1_h,color1_s,color1_v,color2_h,color2_s,color2_v,color3_h,...,portador,transportes,assinado,digital,estadual,infraestrutura,emissor,proibido,plastificar,label
0,aug_imagem_1.jpg,136,1,60,64,4,70,3,252,15,...,0,1,1,1,0,0,0,0,0,0
1,aug_imagem_10.jpg,127,1,60,43,6,60,2,251,18,...,0,1,1,1,0,0,0,0,0,0
2,aug_imagem_100.jpg,97,1,0,0,54,27,14,168,68,...,0,0,1,1,0,0,0,0,0,0
3,aug_imagem_101.jpg,132,1,60,51,5,70,3,252,15,...,0,0,1,1,0,0,0,0,0,0
4,aug_imagem_102.jpg,112,1,0,0,4,15,12,85,70,...,0,1,1,1,0,0,0,0,0,0


In [17]:
#Quantidade de labels diferentes
features_completas['label'].value_counts()

,count
label,
0,288
1,288
2,288
3,288
4,288
5,288


## **Treinamento e avaliação do modelo**

### **Separando features e labels**



In [18]:
# Separação das features (X) e do rótulo (y)

X = features_completas.drop(columns=['label'])
y = features_completas['label']

# 3. Divisão em treino e teste (por exemplo, 80% treino e 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [19]:
y_train

,label
107,0
37,3
269,5
130,2
102,2
...,...
266,3
142,4
284,2
19,5


In [20]:
# Separação das features (X) e do rótulo (y)

# Define 'nome_arquivo' como índice
features_completas = features_completas.set_index('nome_arquivo')

X = features_completas.drop(columns=['label'])
y = features_completas['label']

# 3. Divisão em treino e teste (por exemplo, 80% treino e 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


### **XGBoost**

In [21]:
#4. Criação e treinamento do modelo XGBoost
model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Identifica e remove colunas 'object' do conjunto de treinamento
object_cols_train = X_train.select_dtypes(include=['object']).columns
X_train = X_train.drop(columns=object_cols_train)
X_test = X_test.drop(columns=object_cols_train)

#Treinamento do Modelo
model.fit(X_train, y_train)

#Predição e avaliação
y_pred = model.predict(X_test)

# Acurácia
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acc:.2f}")

# Relatório de classificação
print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [23:23:46] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Acurácia: 0.99

Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        75
           2       1.00      1.00      1.00        55
           3       0.98      0.95      0.97        44
           4       1.00      0.97      0.98        66
           5       0.92      0.98      0.95        50

    accuracy                           0.99       346
   macro avg       0.98      0.98      0.98       346
weighted avg       0.99      0.99      0.99       346



In [22]:

# Identifica erros
erros = y_pred != y_test

# Cria DataFrame com resultados
df_resultados = pd.DataFrame({
    "real": y_test,
    "previsto": y_pred
}, index=X_test.index)

# Filtra os erros
df_erros = df_resultados[erros]

print("\nCasos em que o modelo errou:")
print(df_erros)



Casos em que o modelo errou:
                    real  previsto
nome_arquivo                      
aug_imagem_13.jpg      3         5
aug_imagem_80.jpg      4         5
aug_imagem_140.jpg     4         5
aug_imagem_92.jpg      3         5
aug_imagem_79.jpg      5         3


In [23]:
# Salva o modelo treinado em um arquivo
joblib.dump(model, 'xgboost_cnh_classifier.pkl')
print("Modelo salvo como 'xgboost_cnh_classifier.pkl'")

Modelo salvo como 'xgboost_cnh_classifier.pkl'


### **Random Forest**

In [24]:
model = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acc:.2f}")

print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.99

Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        75
           2       1.00      1.00      1.00        55
           3       0.98      0.95      0.97        44
           4       0.98      0.97      0.98        66
           5       0.96      1.00      0.98        50

    accuracy                           0.99       346
   macro avg       0.99      0.99      0.99       346
weighted avg       0.99      0.99      0.99       346



In [25]:
# Identifica erros
erros = y_pred != y_test

# Cria DataFrame com resultados
df_resultados = pd.DataFrame({
    "real": y_test,
    "previsto": y_pred
}, index=X_test.index)

# Filtra os erros
df_erros = df_resultados[erros]

print("\nCasos em que o modelo errou:")
print(df_erros)


Casos em que o modelo errou:
                    real  previsto
nome_arquivo                      
aug_imagem_19.jpg      3         4
aug_imagem_140.jpg     4         5
aug_imagem_92.jpg      3         5
aug_imagem_234.jpg     4         3


In [26]:
# Salva o modelo treinado em um arquivo
joblib.dump(model, 'random_forest_cnh_classifier.pkl')
print("Modelo salvo como 'random_forest_cnh_classifier.pkl'")

Modelo salvo como 'random_forest_cnh_classifier.pkl'


### **Naive Bayes**

In [27]:
modelo = GaussianNB()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acc:.2f}")

print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))

Acurácia: 0.90

Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       0.99      0.99      0.99        75
           2       0.98      1.00      0.99        55
           3       0.90      0.80      0.84        44
           4       0.98      0.62      0.76        66
           5       0.64      1.00      0.78        50

    accuracy                           0.90       346
   macro avg       0.91      0.90      0.89       346
weighted avg       0.92      0.90      0.90       346



In [28]:
# Identifica erros
erros = y_pred != y_test

# Cria DataFrame com resultados
df_resultados = pd.DataFrame({
    "real": y_test,
    "previsto": y_pred
}, index=X_test.index)

# Filtra os erros
df_erros = df_resultados[erros]

print("\nCasos em que o modelo errou:")
print(df_erros)


Casos em que o modelo errou:
                    real  previsto
nome_arquivo                      
aug_imagem_16.jpg      3         5
aug_imagem_19.jpg      3         4
aug_imagem_117.jpg     4         5
aug_imagem_133.jpg     4         5
aug_imagem_91.jpg      4         5
aug_imagem_80.jpg      4         5
aug_imagem_192.jpg     4         5
aug_imagem_198.jpg     4         5
aug_imagem_160.jpg     4         5
aug_imagem_107.jpg     4         5
aug_imagem_140.jpg     4         5
aug_imagem_146.jpg     3         5
aug_imagem_283.jpg     4         5
aug_imagem_191.jpg     3         5
aug_imagem_49.jpg      4         5
aug_imagem_168.jpg     4         3
aug_imagem_169.jpg     4         5
aug_imagem_268.jpg     4         5
aug_imagem_225.jpg     4         3
aug_imagem_221.jpg     4         5
aug_imagem_69.jpg      3         5
aug_imagem_14.jpg      4         3
aug_imagem_183.jpg     1         2
aug_imagem_154.jpg     3         5
aug_imagem_272.jpg     4         5
aug_imagem_233.jpg     3 

In [29]:
# Salva o modelo treinado em um arquivo
joblib.dump(model, 'naive_bayes_cnh_classifier.pkl')
print("Modelo salvo como 'naive_bayes_cnh_classifier.pkl'")

Modelo salvo como 'naive_bayes_cnh_classifier.pkl'


# **Justificativa da Escolha do modelo - Random Forest**

Para o problema de **classificação automática de documentos de CNH**, foram avaliados três modelos de aprendizado de máquina:
- XGBoost
- Random Forest
- Naive Bayes

A seleção do modelo final foi baseada principalmente na métrica **F1-score**, considerada a mais adequada para cenários de classificação multiclasse quando é necessário equilibrar precisão (precision) e sensibilidade (recall).

Embora a acurácia seja alta em todos os modelos, ela não reflete totalmente a qualidade das predições quando existem classes mais difíceis ou desbalanceadas. Por isso, utilizamos o F1-macro e o F1-weighted, que avaliam o desempenho médio entre as classes de maneira justa e equilibrada.

**O F1-score é a média harmônica entre precisão e recall, sendo uma métrica robusta.**  Assim, o F1-score reflete melhor o desempenho global e evita que classes difíceis fiquem mascaradas por uma acurácia aparentemente alta.

> **Resultados Obtidos**

* **Random Forest:** apresentou F1-macro - 0.99, com excelente desempenho em todas as classes, inclusive nas classes que apresentaram maior dificuldade nos outros modelos (3, 4 e 5).

* **XGBoost:** obteve F1-macro - 0.98, também com desempenho muito alto, mas levemente inferior ao Random Forest.

* **Naive Bayes:** apresentou F1-macro - 0.89, com quedas significativas nas classes 3, 4 e 5, tornando-o inadequado para este problema.